In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r'C:/Users/ashuo/DNS/data/dga_classifier_data/normal_domain_names/opendns-top-domains.txt', header=None, names=['domain_name'])
df2 = pd.read_csv(r'C:/Users/ashuo/DNS/data/dga_classifier_data/normal_domain_names/opendns-top-domains.txt', header=None, names=['domain_name'])
df3 = pd.read_csv(r'C:/Users/ashuo/DNS/data/dga_classifier_data/normal_domain_names/non_malicious_domain_names.txt', header=None, names=['domain_name'])
df3 = df3.drop_duplicates()
df_normal = pd.concat([df1, df2, df3], ignore_index=True)

df_normal = df_normal.drop_duplicates()

df_normal['source'] = 'open_dns'
df_normal['label'] = 0

In [3]:
import os
import pandas as pd
import random

def sample_domains_from_files(directory, sample_size):
    all_domains = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), 'r') as f:
                domains = f.read().splitlines()
                # Check if the file has enough domains to sample
                if len(domains) >= sample_size:
                    # Randomly sample domains from this file
                    sampled_domains = random.sample(domains, sample_size)
                    # Pair each sampled domain with the filename
                    domains_with_source = [(domain, filename) for domain in sampled_domains]
                    all_domains.extend(domains_with_source)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_domains, columns=['domain_name', 'source'])
    
    return df

# Use the function
directory = r'C:/Users/ashuo/DNS/data/dga_classifier_data/dga_domain_names'  # replace with your actual directory
sample_size = 275
df_dga = sample_domains_from_files(directory, sample_size)

In [4]:
df_dga.shape
df_dga['label'] = 1

In [5]:
df = pd.concat([df_normal, df_dga], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
#df = df.drop(['source'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39793 entries, 0 to 39792
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   domain_name  39793 non-null  object
 1   source       39793 non-null  object
 2   label        39793 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 932.8+ KB


In [7]:
from numpy import double
import tldextract
from collections import Counter
import math
from tld import get_tld

dataset = []

def calculate_entropy(domain):
    # Count the frequency of each character in the domain
    freq = Counter(domain)
    
    # Calculate the probability of each character
    domain_length = double(len(domain))
    probabilities = [float(freq[char]) / domain_length for char in set(domain)]
    
    # Calculate the entropy using the Shannon entropy formula
    entropy = -sum(p * math.log2(p) for p in probabilities)
    
    return entropy

def calculate_average_entropy_of_subdomains(ext):

    if ext.subdomain == '':
        return 0.0
    # Split the domain into subdomains using the dot separator
    subdomains = ext.subdomain.split('.')
    
    # Calculate the entropy of each subdomain
    subdomain_entropies = [calculate_entropy(sub) for sub in subdomains]
    
    # Calculate the average entropy of subdomains
    avg_entropy = sum(subdomain_entropies)/double(len(subdomain_entropies))
    
    return avg_entropy

def calculate_alphanumeric_ratio(domain):
    # Count the number of alphanumeric characters (lowercase alphabetic or numeric)
    alphanumeric_count = sum(1 for char in domain if char.isalnum() and char.islower())
    
    # Count the total number of characters in the domain
    total_characters = double(len(domain))

    alphanumeric_ratio = alphanumeric_count / total_characters
    
    return alphanumeric_ratio

def count_number_of_subdomains(ext):
    if ext.subdomain == '':
        return 0.0

    subdomains = ext.subdomain.split('.')

    return len(subdomains)


def calculate_average_subdomain_length(ext):
    if ext.subdomain == '':
        return 0.0
    # Split the domain into subdomains using the dot separator
    subdomains = ext.subdomain.split('.')

    subdomains_to_consider = subdomains

    # Calculate the length of each subdomain
    subdomain_lengths = [len(sub) for sub in subdomains_to_consider]

    avg_subdomain_length = sum(subdomain_lengths)/double(len(subdomain_lengths))

    return avg_subdomain_length

def calculate_numeric_ratio(domain):
    # Calculate the number of numeric characters in the domain
    numeric_count = double(sum(1 for char in domain if char.isdigit()))
    
    # Calculate the total length of the domain
    total_length = double(len(domain))
    
    if(total_length == 0):
        return 0.0
    
    numeric_ratio = numeric_count / total_length
    
    return numeric_ratio

def calculate_special_char_ratio(domain):
    # Count the number of special characters in the domain excluding '.'
    special_char_count = sum(1 for char in domain if not (char.isalnum() or char == '.'))

    # Calculate the total length of the domain excluding '.'
    total_length_excluding_dot = double(len([char for char in domain if char != '.']))

    # Calculate the special character ratio
    if total_length_excluding_dot > 0:
        special_char_ratio = double(special_char_count)/double(total_length_excluding_dot)
    else:
        special_char_ratio = 0.0  # Set ratio to 0 if the domain has no characters excluding '.'

    return special_char_ratio

def count_digits(domain):
    # Initialize a counter for digits
    digit_count = 0

    # Iterate through each character in the domain name
    for char in domain:
        if char.isdigit():  # Check if the character is a digit
            digit_count += 1  # Increment the counter for each digit found

    return digit_count

def count_hyphens(domain):
    hyphen_count = 0

    for char in domain:
        if char == '-':  
            hyphen_count += 1  

    return hyphen_count

def count_vowels(domain):
    vowels = {'a', 'e', 'i', 'o', 'u'}
    domain_lower = domain.lower()
    vowel_count = sum(1 for char in domain_lower if char in vowels)
    return vowel_count

def calculate_repeated_chars_ratio(subdomain):
    subdomain_lower = subdomain.lower()
    total_chars = len(subdomain_lower)
    unique_chars = len(set(subdomain_lower))
    repeated_chars = total_chars - unique_chars
    repeated_chars_ratio = double(repeated_chars)/total_chars if total_chars > 0 else 0.0

    return repeated_chars_ratio

def calculate_consecutive_digits_ratio(domain):
    # Extract the domain part without the valid public suffix (TLD)
    ext = tldextract.extract(domain)
    domain_without_tld = ext.domain

    # Convert the domain name to lowercase for consistency
    domain_lower = domain_without_tld.lower()

    # Initialize variables for counting consecutive digits and total characters
    consecutive_digits_count = 0
    total_chars = len(domain_lower)

    # Iterate through the domain characters to count consecutive digits
    i = 0
    while i < total_chars - 1:
        if domain_lower[i].isdigit() and domain_lower[i + 1].isdigit():
            consecutive_digits_count += 1
            # Skip to the next non-digit character
            while i < total_chars - 1 and domain_lower[i].isdigit():
                i += 1
        else:
            i += 1
    print(consecutive_digits_count)
    # Calculate the ratio of consecutive digits
    consecutive_digits_ratio = consecutive_digits_count / total_chars if total_chars > 0 else 0

    return consecutive_digits_ratio

def count_underscore(domain):
    underscore_count = 0

    for char in domain:
        if char == '_':  
            underscore_count += 1  

    return underscore_count

def get_feature_rich_row(domain):
    features = {}

    # Split the domain into subdomains and TLD (Top-Level Domain)
    ext = tldextract.extract(domain)
    
    # Full domain
    #features['full_domain'] = domain
    
    # Top-Level Domain
    features['tld'] = ext.suffix if ext.suffix else 'not_present'    

    features['has_tld'] = 1 if ext.suffix else 0
    
    # Domain and Subdomain lengths
    features['full_domain_length'] = len(domain)
    features['domain_length'] = len(ext.domain)
    
    # Check if the domain has a subdomain
    features['has_subdomain'] = 1 if ext.subdomain else 0

    features['subdomain_length'] = len(ext.subdomain)

    #Calculating the number of subdomains
    features['subdomains_count'] = count_number_of_subdomains(ext)
    
    #Calculating the average length of subdomains
    features['avg_subdomain_length'] = calculate_average_subdomain_length(ext)
    
    #Calculating the entropy of the domain name
    features['entropy_of_domain'] = calculate_entropy(domain)
    
    #Caluclating the average entropy of subdomains
    features['avg_entropy_of_subdomains'] = calculate_average_entropy_of_subdomains(ext)
    
    #Calculating the ratio of alphanumeric characters in the domain name
    features['alphanumeric_ratio'] = calculate_alphanumeric_ratio(domain)
    
    #calculating the ratio of numeric characters in the domain name
    features['numeric_ratio'] = calculate_numeric_ratio(domain)
    
    #Calculating the ratio of special characters in the domain name
    features['special_char_ratio'] = calculate_special_char_ratio(domain)

    features['underscore_ratio'] = domain.count('_') / len(domain)

    number_of_digits = count_digits(domain)

    features['contains_digit']  = 1 if number_of_digits > 0 else 0

    length = double(len(ext.domain) +len(ext.subdomain))
    
    features['digit_ratio'] = number_of_digits / length if length > 0 else 0.0

    features['hyphen_ratio'] = (count_hyphens(domain)) / length if length > 0 else 0.0

    features['underscore_ratio'] = (count_underscore(domain))/ length if length > 0 else 0.0

    features['vowel_ratio'] =  (count_vowels(domain))/ length if length > 0 else 0.0

    features['repeated_chars_ratio'] = calculate_repeated_chars_ratio(ext.domain)

    return features



for index, row in df.iterrows():
    domain_name = row['domain_name']
    label = row['label']
    feature_rich_row = get_feature_rich_row(domain_name)
    feature_rich_row['label'] = label
    dataset.append(feature_rich_row)

dataset_df = pd.DataFrame(dataset)

#print(dataset_df)

dataset_df.to_csv(r'C:/Users/ashuo/DNS/data/dga_classifier.csv', index=False)

df = dataset_df


In [8]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['tld'])
df = encoder.fit_transform(df)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39793 entries, 0 to 39792
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tld_0                      39793 non-null  int64  
 1   tld_1                      39793 non-null  int64  
 2   tld_2                      39793 non-null  int64  
 3   tld_3                      39793 non-null  int64  
 4   tld_4                      39793 non-null  int64  
 5   tld_5                      39793 non-null  int64  
 6   tld_6                      39793 non-null  int64  
 7   tld_7                      39793 non-null  int64  
 8   tld_8                      39793 non-null  int64  
 9   has_tld                    39793 non-null  int64  
 10  full_domain_length         39793 non-null  int64  
 11  domain_length              39793 non-null  int64  
 12  has_subdomain              39793 non-null  int64  
 13  subdomain_length           39793 non-null  int

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = df.drop('label',axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
import pickle

# Save the encoder and scaler to files
with open(r'C:/Users/ashuo/DNS/models/dga_tld_encoder.pkl', 'wb') as encoder_file:
    pickle.dump(encoder, encoder_file)

with open(r'C:/Users/ashuo/DNS/models/dga_feature_scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


In [12]:
# # Assuming your new input data is in a variable named 'new_data'

# # For binary encoding:
# new_data_encoded = loaded_encoder.transform(new_data['tld_column'])

# # For feature scaling:
# scaled_new_features = loaded_scaler.transform(new_data)  # Replace 'new_data' with your input feature matrix

In [13]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize models
svm_model = SVC()
rf_model = RandomForestClassifier(random_state=42)
gbt_model = GradientBoostingClassifier(random_state=42)

# Train models
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
gbt_model.fit(X_train, y_train)

# Make predictions
svm_pred = svm_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
gbt_pred = gbt_model.predict(X_test)

# Calculate evaluation metrics
models = [svm_model, rf_model, gbt_model]
metrics = []

for model in models:
    accuracy = accuracy_score(y_test, model.predict(X_test))
    precision = precision_score(y_test, model.predict(X_test))
    recall = recall_score(y_test, model.predict(X_test))
    f1 = f1_score(y_test, model.predict(X_test))
    metrics.append([accuracy, precision, recall, f1])

# Create a DataFrame for the table
results_df = pd.DataFrame(metrics, columns=['Accuracy', 'Precision', 'Recall', 'F-Score'], index=['SVM', 'RF', 'GBT'])

print("Model Evaluation Metrics:")
print(results_df)


Model Evaluation Metrics:
     Accuracy  Precision    Recall   F-Score
SVM  0.840181   0.869848  0.791624  0.828894
RF   0.840684   0.863234  0.801131  0.831023
GBT  0.830004   0.859326  0.780062  0.817778


In [20]:
# Load saved models
with open(r'C:/Users/ashuo/DNS/models/dga_svm_model.pkl', 'wb') as svm_file:
    pickle.dump(svm_model, svm_file)

with open(r'C:/Users/ashuo/DNS/models/dga_rf_model.pkl', 'wb') as rf_file:
    pickle.dump(rf_model, rf_file)

with open(r'C:/Users/ashuo/DNS/models/dga_gbt_model.pkl', 'wb') as gbt_file:
    pickle.dump(gbt_model, gbt_file)